# Model/Reanalysis SST

## Data Sources and Description

There are lots of sst products - either derived from sat data or gridded from insitu data or modelled or some combination.

Best to focus on just a few: Sat product (OISST), Highres product (MUR) and non-US product (UKMet)

Retrieve regional subsets of SST including any error and ice information (these should be in seperate datasets?).

An ultimate dataset would have each drifter/ship/other point collocated with the one datapoint from the models)


In [1]:
#avhrr only data from ncei - this is highres oi V2.1 data - 0.25deg resolution
erddap_path_avhrr_only="https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst2Agg" 
erddap_path_hroisst="https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst21Agg" #newer v2.1

#avhrr+amsr data is available for 2002-2011 from ncei
erddap_path_avhrramsre="https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst2AmsrAgg" #version 2... not 2.1 if one exists

#gh1sst data - ROMS from JPL - .01deg
erddap_path_ghsst="https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplG1SST" #project not encouraged via coastwatch - push for MUR
# JPL MUR - OI using wavelets... theres a .01deg and .25 (not on coastwatch erddap)
erddap_path_ghsst="https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplMURSST41" 
# https://podaac-tools.jpl.nasa.gov/drive/files/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR25/v4.2

#blended product from Remote Sensing Systems http://www.remss.com/ - does not span entire time series
erddap_path_blended = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/erdG1ssta1day"

#gh1sst data - UKMET 0.05deg resolution
erddap_UKMET= "https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplUKMO_OSTIAv20"


## General Modelset Details

(PSL - https://psl.noaa.gov/data/gridded/tables/sst.html)  
(JPL - https://podaac.jpl.nasa.gov)

NCEP OISSTV2 - 0.25deg resolution (0:360 degE) 
+ lots of good details on this page (https://www.ncdc.noaa.gov/oisst/optimum-interpolation-sea-surface-temperature-oisst-v21 and https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc:C01606/html)  
+ (podaac doesn't have it?)

JPL MUR - 0.01deg resolution https://podaac.jpl.nasa.gov/dataset/MUR-JPL-L4-GLOB-v4.1 (there is a 0.25deg version too... v4.2) : uses nighttime skin/subkin temps and "The ice concentration data are from the archives at the EUMETSAT Ocean and Sea Ice Satellite Application Facility (OSI SAF) High Latitude Processing Center and are also used for an improved SST parameterization for the high-latitudes."

UKMET - (via jpl) - 0.05deg resolution (-180:180 degE)

Canadian Model - 0.1 or 0.2deg resolution (not on coastwatch erddap) - https://podaac.jpl.nasa.gov/dataset/CMC0.1deg-CMC-L4-GLOB-v3.0?ids=Keywords:Processing%20Levels&values=Oceans:Ocean%20Temperature::4%20-%20Gridded%20Model%20Output)

REMSS V5 SST (no in-situ data used)
https://podaac.jpl.nasa.gov/dataset/MW_OI-REMSS-L4-GLOB-v5.0?ids=Keywords:Processing%20Levels&values=Oceans:Ocean%20Temperature::4%20-%20Gridded%20Model%20Output

and a subskin AMSRE satellite retrieval dataset - https://podaac.jpl.nasa.gov/dataset/AMSR2-REMSS-L3U-v8a?ids=Keywords&values=Oceans:Ocean%20Temperature:Sea%20Surface%20Temperature  
or  
maybe use the AMSRE+AVHRR datasets?  
or  
a modis dataset? (https://coastwatch.pfeg.noaa.gov/erddap/griddap/erdMH1sstd1dayR20190SQ)


## SUBSET FOR 2010+ ONLY

***Dataset/Programming Note:***

Xarray's `open_dataset` function lazyloads and allows exploration, `load_dataset` pulls it to memory and `da.sel().load()` will load a subset to memory.  If the data has to be retrieved every datapoint, that will take forever.  Local loading of data is more convenient by far.

**ADVICE:**
Either call via erddap a subset location over a year and process expected in-situ data (works well for argos_NRT datasets) _or_ download dataset locally to work from so there is no network call.

In [2]:
import xarray as xa

In [ ]:
#four times a year breakdown - run on system with strong memory or network speed
xa_UKMET = xa.open_dataset(erddap_UKMET)

In [ ]:
for i in range(2010,2011,1):
    print(i)
    xa_UKMET_subset = xa_UKMET.sel(time=slice(str(i)+'-01-01',str(i)+'-3-30'),
                                     latitude=slice(57,90),longitude=slice(-180,-150)).load()
    xa_UKMET_subset.to_netcdf('NBS_CK_UKMET_'+str(i)+'.0.nc')

    xa_UKMET_subset = xa_UKMET.sel(time=slice(str(i)+'-04-01',str(i)+'-6-30'),
                                     latitude=slice(57,90),longitude=slice(-180,-150)).load()
    xa_UKMET_subset.to_netcdf('NBS_CK_UKMET_'+str(i)+'.1.nc')
    
    xa_UKMET_subset = xa_UKMET.sel(time=slice(str(i)+'-07-01',str(i)+'-9-30'),
                                     latitude=slice(57,90),longitude=slice(-180,-150)).load()
    xa_UKMET_subset.to_netcdf('NBS_CK_UKMET_'+str(i)+'.2.nc')    
    
    xa_UKMET_subset = xa_UKMET.sel(time=slice(str(i)+'-10-01',str(i)+'-12-31'),
                                     latitude=slice(57,90),longitude=slice(-180,-150)).load()
    xa_UKMET_subset.to_netcdf('NBS_CK_UKMET_'+str(i)+'.3.nc')    

In [ ]:
xa_UKMET_subset

In [4]:
#open_dataset vs load_dataset
erddap_path_hroisst="http://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst21Agg" #newer
xa_HROISST = xa.open_dataset(erddap_path_hroisst)
for i in range(2010,2021,1):
    print(i)
    xa_HROISST_subset = xa_HROISST.sel(time=slice(str(i)+'-01-01',str(i)+'-12-31'),
                                     latitude=slice(55,90),longitude=slice(170,210)).load()

    xa_HROISST_subset.to_netcdf('NBS_CK_HROISST_'+str(i)+'.nc')

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [ ]:
xa_HROISST_subset

In [3]:
xa_ghsst = xa.open_dataset(erddap_path_ghsst)

In [ ]:
xa_ghsst_subset= xa_ghsst['analysed_sst'].sel(time=slice('2010-2','2010-2'),latitude=slice(55,66),longitude=slice(-180,-150)).load()

In [ ]:
xa_ghsst_subset.to_netcdf('NBS_CK_MUR_'+str(i)+'.1.nc')

In [ ]:
#monthly breakdown
xa_ghsst = xa.open_dataset(erddap_path_ghsst)
for i in range(2010,2021,1):
    print(i)
    xa_ghsst_subset = xa_ghsst['analysed_sst'].sel(time=slice(str(i)+'-1'),
                                     latitude=slice(57,90),longitude=slice(-180,-150)).load()

    #xa_UKMET_subset.load()
    xa_ghsst_subset.to_netcdf('NBS_CK_MUR_'+str(i)+'.1.nc')

In [ ]:
xa_ghsst_subset

In [ ]:
xa_ghsst = xa.open_dataset(erddap_path_ghsst)


In [ ]:
xa_ghsst
xa_ghsst_subset = xa_ghsst['analysed_sst'].sel(time=slice(str(i)+'-06-01',str(i)+'-12-31'),
                                     latitude=slice(57,90),longitude=slice(170,210))